## RNN1

**Input:** Matrix of Word embeddings -> each element is an embedding for a word

**Output:** Single final output at the end of the network -> we can call it the global context vector (can test GloVe method and get it ready as another alternative function for Word2Vec), we will call it h-1



## RNN2  
#### ( input X:the sequence vector of characters X2 -> each element is a character)  
(hidden input h-1 : is the output of RNN1)
|  (output  vector of labels , Yi corresponds as a tashkela to Xi character

**Input:** Matrix of Character embeddings -> each element is an embedding for a character

**Output:** Matrix of labels -> each element is a label for a character

In [1]:
import torch
from torch import nn

In [5]:
%pip install --upgrade pip
# %pip install numpy --upgrade
%pip install numpy --upgrade --ignore-installed
%pip install tensorflow
%pip install keras
%pip install gensim
import re
from collections import Counter
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk, re
from keras.preprocessing.text import Tokenizer
from datetime import datetime
from gensim.models import *
import logging
# from rnn_utils import *
%matplotlib inline


''' D_NAMES: This is a list containing names of various Arabic diacritics. Each
 element of the list represents a specific diacritic type. '''
D_NAMES = ['Fathatan', 'Dammatan', 'Kasratan', 'Fatha', 'Damma', 'Kasra', 'Shadda', 'Sukun']

##############################################################################################

''' NAME2DIACRITIC: This uses a dictionary comprehension to create a mapping 
from diacritic names to their corresponding Unicode characters.'''
NAME2DIACRITIC = dict((name, chr(code)) for name, code in zip(D_NAMES, range(0x064B, 0x0653)))

##############################################################################################

''' DIACRITIC2NAME: This is the inverse of the previous dictionary.'''
DIACRITIC2NAME = dict((code, name) for name, code in NAME2DIACRITIC.items())

##############################################################################################

''' ARABIC_DIACRITICS: This creates a frozenset containing the Unicode
 characters of all the diacritics.'''
ARABIC_DIACRITICS = frozenset(NAME2DIACRITIC.values())


# Remove all standard diacritics from the text, leaving the letters only.
def clear_diacritics(text):
    assert isinstance(text, str)
    return ''.join([l for l in text if l not in ARABIC_DIACRITICS])


# Return the diacritics from the text while keeping their original positions.
def extract_diacritics(text):
    assert isinstance(text, str)
    diacritics = []
    classes = []
    temp = ''
    for i in range(1, len(text)):
        temp = ''
        if text[i] in ARABIC_DIACRITICS:
            if text[i-1] == NAME2DIACRITIC['Shadda']:
                diacritics[-1] = (DIACRITIC2NAME[text[i-1]], DIACRITIC2NAME[text[i]])
                temp = (DIACRITIC2NAME[text[i-1]], DIACRITIC2NAME[text[i]])
                if (temp == ('Shadda', 'Fatha')):
                    classes.pop()
                    classes.append(8)
                elif (temp == ('Shadda', 'Fathatan')):
                    classes.pop()
                    classes.append(9)
                elif (temp == ('Shadda', 'Damma')):
                    classes.pop()
                    classes.append(10)
                elif (temp == ('Shadda', 'Dammatan')):
                    classes.pop()
                    classes.append(11)
                elif (temp == ('Shadda', 'Kasra')):
                    classes.pop()
                    classes.append(12)
                elif (temp == ('Shadda', 'Kasratan')):
                    classes.pop()
                    classes.append(13)
            else:
                diacritics.append(DIACRITIC2NAME[text[i]])
                temp = DIACRITIC2NAME[text[i]]
                if (temp == 'Fatha'):
                    classes.append(0)
                elif (temp == 'Fathatan'):
                    classes.append(1)
                elif (temp == 'Damma'):
                    classes.append(2)
                elif (temp == 'Dammatan'):
                    classes.append(3)
                elif (temp == 'Kasra'):
                    classes.append(4)
                elif (temp == 'Kasratan'):
                    classes.append(5)
                elif (temp == 'Sukun'):
                    classes.append(6)
                elif (temp == 'Shadda'):
                    classes.append(7)
        elif text[i - 1] not in ARABIC_DIACRITICS:
            diacritics.append('')
            classes.append(14)
        
    if text[-1] not in ARABIC_DIACRITICS:
        diacritics.append('')
        classes.append(14)
    return diacritics, classes


def extract_arabic_words2(text):
    arabic_pattern = re.compile('[\u0600-\u06FF]+')
    arabic_matches = arabic_pattern.findall(text)
    result = ' '.join(arabic_matches)
    processed_text = re.sub(r'[؛،\.]+', '', result)
    final_processed_text = re.sub(r'\s+', ' ', processed_text)
    return final_processed_text


input_file_path = "train.txt"  # Replace with your input file path
with open(input_file_path, "r", encoding="utf-8") as input_file:
    input_text = input_file.read()

arabic_words = extract_arabic_words2(input_text)

output_words = clear_diacritics(arabic_words)
words = output_words.split()
words2 = arabic_words.split()
words_array = [list(word) for word in words]
words_array2 = [list(word2) for word2 in words2]

output_without_spaces = arabic_words.replace(" ", "")
output_without_spaces2 = output_words.replace(" ", "")
array_of_chars = [char for char in output_without_spaces]
_,classes_extraction = extract_diacritics (output_without_spaces)


num_feature = 100
min_word_count = 1
num_thread = 5
window_size = 10
down_sampling = 0.001
iteration = 20

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model_fastText = FastText(words_array,
                        vector_size=num_feature,
                        window=window_size,
                        min_count=min_word_count,
                        workers=num_thread)


j=0
chars =[]
char_vectors =[]
char_classes=[]
for word in words_array:
  i=i+1
  for char in word:
    chars.append(char)
    char_classes.append(classes_extraction[j])
    vector = model_fastText.wv[char]
    char_vectors.append(vector)
    j=j+1

print (j)
print(chars[1])
print(char_classes[1])
print(char_vectors[1])


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipympl 0.9.3 requires matplotlib<4,>=3.4.0, but you have matplotlib 3.3.4 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed

In [31]:
# class RNN_Model(nn.Module):
#     def __init__(self, input_size, output_size, hidden_dim, n_layers):
#         super(RNN_Model, self).__init__()
        
#         # Defining some parameters
#         self.hidden_dim = hidden_dim
#         self.n_layers = n_layers
        
#         #Defining the layers
#         # RNN Layer
#         self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
#         # Fully connected layer
#         self.fc = nn.Linear(hidden_dim, output_size)
        
#     def forward(self, x, h_prev):
            
#             batch_size = x.size(0)
            
#             #initialize hidden state with h_prev
#             hidden = h_prev
            
#             #make sure hidden state dimensions are correct
#             hidden = hidden.view(self.n_layers, batch_size, self.hidden_dim)

#             # Passing in the input and hidden state into the model and obtaining outputs
#             out, hidden = self.rnn(x, hidden)
#             # Reshaping the outputs such that it can be fit into the fully connected layer
#             out = out.contiguous().view(-1, self.hidden_dim)
#             out = self.fc(out)
            
#             return out, hidden

import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        
        # RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0):
        # Forward pass through the RNN
        out, hn = self.rnn(x, h0)
        
        # Select the output from the last time step
        out = out[:, -1, :]
        
        # Fully connected layer
        out = self.fc(out)
        
        return out, hn

# Example usage
input_size = 10  # Adjust based on your input features
hidden_size = 32
output_size = 2  # Adjust based on your task (e.g., classification)

# Initialize the model
rnn_model = SimpleRNN(input_size, hidden_size, output_size)

# Example input (batch_size=3, sequence_length=5, input_size=10)
sample_input = torch.randn(3, 5, 10)

# Initial hidden state (num_layers * num_directions, batch, hidden_size)
h0 = torch.zeros(1, 3, hidden_size)

# Forward pass through the RNN
output, final_hidden = rnn_model(sample_input, h0)

# Print the output shape and final hidden state shape
print("Output Shape:", output.shape)
print("Final Hidden State Shape:", final_hidden.shape)
#####################
def train_model(model, train_loader):
    """
    Function for training the model
    """
    # define the optimization
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    # define the loss function
    criterion = nn.CrossEntropyLoss()
    # epochs
    epochs = 10
    # loop over the epochs
    for epoch in range(epochs):
        # initialize the hidden state
        h0 = torch.zeros(1, 3, hidden_size)
        # loop over the dataset
        for inputs, labels in train_loader:
            # zero the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat, h0 = model(inputs, h0)
            # calculate loss
            loss = criterion(yhat, labels)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
        # print the loss
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
#####################

def evaluate_model(model, test_loader):
    """
    Function for evaluating the model
    """
    # initialize the hidden state
    h0 = torch.zeros(1, 3, hidden_size)
    # initialize the accuracy
    correct = 0
    total = 0
    # deactivating autograd
    with torch.no_grad():
        # loop over the test dataset
        for inputs, labels in test_loader:
            # compute the model output
            yhat, h0 = model(inputs, h0)
            # get predictions from the maximum value
            _, predicted = torch.max(yhat.data, 1)
            # update total
            total += labels.size(0)
            # update correct
            correct += (predicted == labels).sum().item()
    # compute the accuracy
    accuracy = 100 * correct / total
    # print the accuracy
    print(f'Accuracy: {accuracy:.2f}')
    
# # evaluate the model
# evaluate_model(rnn_model, test_loader)
# #####################
# def predict(model, test_loader):
#     """
#     Function for predicting the results
#     """
#     # initialize the hidden state
#     h0 = torch.zeros(1, 3, hidden_size)
#     # initialize the prediction list
#     predictions = []
#     # deactivating autograd
#     with torch.no_grad():
#         # loop over the test dataset
#         for inputs, labels in test_loader:
#             # compute the model output
#             yhat, h0 = model(inputs, h0)
#             # get predictions from the maximum value
#             _, predicted = torch.max(yhat.data, 1)
#             # append the batch prediction results
#             predictions.append(predicted.numpy())
#     # combine all the batch prediction results
#     predictions = np.concatenate(predictions)
#     # print the predictions
#     print(predictions)



# Example usage
input_size = 10  # Adjust based on your input features
hidden_size = 32
output_size = 2  # Adjust based on your task (e.g., classification)

# Initialize the model
rnn_model = SimpleRNN(input_size, hidden_size, output_size)

# Example input (batch_size=3, sequence_length=5, input_size=10)
sample_input = torch.randn(3, 5, 10)

# Initial hidden state (num_layers * num_directions, batch, hidden_size)
h0 = torch.zeros(1, 3, hidden_size)

# Forward pass through the RNN
output, final_hidden = rnn_model(sample_input, h0)

# Print the output shape and final hidden state shape
print("Output Shape:", output.shape)
print("Final Hidden State Shape:", final_hidden.shape)

#create dummy train and test datasets with 3 samples, 5 timesteps, and 10 features
train = torch.randn(3, 5, 10)
test = torch.randn(3, 5, 10)

#loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=3)
test_loader = torch.utils.data.DataLoader(test, batch_size=3)

#train the model
train_model(rnn_model, train_loader)

#evaluate the model
evaluate_model(rnn_model, test_loader)



Output Shape: torch.Size([3, 2])
Final Hidden State Shape: torch.Size([1, 3, 32])
Output Shape: torch.Size([3, 2])
Final Hidden State Shape: torch.Size([1, 3, 32])


ValueError: too many values to unpack (expected 2)

Encoder model

In [3]:

# class Encoder(nn.Module):
#     """Encoder layer to encode a sequence to a hidden state"""

#     def __init__(self, input_size, hidden_size, num_layers, flavor):
#         """
#         Parameters
#         ----------
#         input_size: int
#         hidden_size: int
#             Number of hidden units in the RNN model
#         num_layers: int
#             Number of layers in the RNN model
#         flavor: str
#             Takes 'rnn', 'lstm', or 'gru' values.
#         """
#         super(Encoder, self).__init__()
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.flavor = flavor

#         if flavor == 'rnn':
#             self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         elif flavor == 'lstm':
#             self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         elif flavor == 'gru':
#             self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
#         else:
#             raise ValueError("Invalid flavor. Choose 'rnn', 'lstm', or 'gru'.")

#     def forward(self, x, hidden):
#         """
#         Forward pass of the encoder layer.

#         Parameters
#         ----------
#         x: torch.Tensor
#             Input sequence tensor of shape (batch_size, sequence_length, input_size)
#         hidden: torch.Tensor
#             Initial hidden state tensor of shape (num_layers, batch_size, hidden_size)

#         Returns
#         -------
#         output: torch.Tensor
#             Output tensor of shape (batch_size, sequence_length, hidden_size)
#         hidden: torch.Tensor
#             Updated hidden state tensor of shape (num_layers, batch_size, hidden_size)
#         """
#         output, hidden = self.rnn(x, hidden)
#         return output, hidden

#     def init_hidden(self, batch_size):
#         """
#         Initialize the hidden state tensor.

#         Parameters
#         ----------
#         batch_size: int
#             Size of the batch

#         Returns
#         -------
#         hidden: torch.Tensor
#             Initial hidden state tensor of shape (num_layers, batch_size, hidden_size)
#         """
#         if self.flavor == 'lstm':
#             return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
#                     torch.zeros(self.num_layers, batch_size, self.hidden_size))
#         else:
#             return torch.zeros(self.num_layers, batch_size, self.hidden_size)


class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, flavor):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.flavor = flavor

        if flavor == 'rnn':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        elif flavor == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif flavor == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid flavor. Choose 'rnn', 'lstm', or 'gru'.")

    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        if self.flavor == 'lstm':
            return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                    torch.zeros(self.num_layers, batch_size, self.hidden_size))
        else:
            return torch.zeros(self.num_layers, batch_size, self.hidden_size)


In [10]:
# class Tashkeel_Model(nn.Module):
#     """ encoder layer + RNN layer"""
#     def __init__(self, input_size, output_size, hidden_size, num_layers, flavor):
#         super(Tashkeel_Model, self).__init__()
#         self.encoder = Encoder(input_size, hidden_size, num_layers, flavor)
#         self.rnn = RNN_Model(input_size, output_size, hidden_size, num_layers)

#     def forward(self, x):
#         print("x shape is ", x.shape)
#         #forward pass of the encoder
#         h_minus_1 = self.encoder.init_hidden(x.shape[0])
#         print("h_minus_1 shape is ", h_minus_1.shape)
#         out, h_minus_1 = self.encoder(x, h_minus_1)
#         print("encoder output shape is ", h_minus_1.shape)
#         #adjust the shape of the output of the encoder
#         h_minus_1 = h_minus_1.view(h_minus_1.shape[1], h_minus_1.shape[0], h_minus_1.shape[2])
#         out, hidden = self.rnn(out, h_minus_1)
#         return out, hidden

class Tashkeel_Model(nn.Module):
    def __init__(self, word_input_size, char_input_size, output_size, hidden_size, num_layers, flavor):
        super(Tashkeel_Model, self).__init__()
        self.word_encoder = Encoder(word_input_size, hidden_size, num_layers, flavor)
        self.rnn = RNN_Model(char_input_size, output_size, hidden_size, num_layers)

    def forward(self, words, chars):
        # Forward pass of the word encoder
        h_minus_1 = self.word_encoder.init_hidden(words.shape[0])
        out, h_minus_1 = self.word_encoder(words, h_minus_1)
        # Adjust the shape of the output of the encoder
        # Forward pass of the RNN
        out, hidden = self.rnn(chars, h_minus_1)
        return out, hidden


def train_tashkeel_model(model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            # Move the data to the device that is used
            data = data.to(device)
            target = target.to(device)

            # Forward pass the Encoder
            output, _ = model(data)

            # Calculate the batch loss
            loss = criterion(output, target)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print loss for progress
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))





In [11]:
import torch
from torch.utils.data import Dataset

class TashkeelDataset(Dataset):
    def __init__(self, word_data, char_data, labels):
        self.word_data = word_data
        self.char_data = char_data
        self.labels = labels

    def __len__(self):
        return len(self.word_data)

    def __getitem__(self, idx):
        word_sequence = self.word_data[idx]
        char_sequence = self.char_data[idx]
        label = self.labels[idx]

        return torch.tensor(word_sequence), torch.tensor(char_sequence), torch.tensor(label)


In [15]:
# Example usage
word_size = 20
char_size = 10
hidden_size = 128
output_size = 300  # Adjust based on the number of classes for diacritics or the dimension of your embeddings
num_layers = 1
flavor = 'rnn'

tashkeel_model = Tashkeel_Model(word_size, char_size, output_size, hidden_size, num_layers, flavor)

dummy_word_data = torch.randn(32, 10, word_size)
dummy_char_data = torch.randn(32, 10, char_size)
dummy_labels = torch.randn(32, 10, output_size)

dummy_dataset = TashkeelDataset(dummy_word_data, dummy_char_data, dummy_labels)
dummy_loader = torch.utils.data.DataLoader(dummy_dataset, batch_size=32, shuffle=True)

criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
optimizer = torch.optim.Adam(tashkeel_model.parameters(), lr=0.001)

train_tashkeel_model(tashkeel_model, dummy_loader, criterion, optimizer, num_epochs=10)



# Define your DataLoader for training, including the input data

# train_dataset = TashkeelDataset(word_data, char_data, labels)
# #! TODO : add the target data
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
# optimizer = optim.Adam(tashkeel_model.parameters(), lr=0.001)
# train_tashkeel_model(tashkeel_model, train_loader, criterion, optimizer, num_epochs=10)
# dummy



<ipython-input-11-60d140dea7f9>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(word_sequence), torch.tensor(char_sequence), torch.tensor(label)


ValueError: too many values to unpack (expected 2)

In [ ]:
# Example usage:
input_size = 300  # Adjust based on the dimension of your word embeddings
hidden_dim = 128
n_layers = 2  # Number of RNN layers

word_encoder = WordEncoder(input_size, hidden_dim, n_layers)

# Assuming 'input_sequence' is your input tensor
output_sequence, hidden_state = word_encoder(input_sequence)

In [6]:
import fasttext


test_input_sentence = 'السلام عليكم ورحمة الله وبركاته'

test_input_sentence_words = test_input_sentence.split(' ')
print(test_input_sentence_words)

# load the embedding model
model = fasttext.load_model('word_embedding_model.bin')

# get the embedding of the input sentence
test_input_sentence_embedding = [
    model[word] for word in test_input_sentence_words
]
print(test_input_sentence_embedding)

# convert the embedding to tensor
test_input_sentence_embedding = torch.tensor(test_input_sentence_embedding)
print(test_input_sentence_embedding)

# reshape the tensor
test_input_sentence_embedding = test_input_sentence_embedding.view(1, -1, 100)
print(test_input_sentence_embedding)

# 





100
[[ 0.042906   -0.02756045 -0.15988487 ...  0.06463267 -0.00134799
  -0.07523397]
 [-0.17331032  0.11529611 -0.34257153 ...  0.02257426  0.08717281
   0.01006125]
 [ 0.05427927 -0.08796314 -0.3034795  ...  0.06067655 -0.19971474
   0.02589632]
 ...
 [-0.01609128  0.14161943 -0.5717865  ...  0.41028932 -0.2221118
  -0.5745774 ]
 [-0.12516408  0.20515387 -0.6757796  ...  0.16081104  0.12018422
  -0.57703876]
 [-0.12468552 -0.08162142 -0.47292602 ...  0.24188764 -0.19347623
  -0.3513778 ]]


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# class WordEncoder(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(WordEncoder, self).__init__()
#         self.hidden_size = hidden_size

#         # Define the Encoder layer as bidirectional LSTM
#         self.encoder = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)

#         # Define the output layer for reconstruction
#         self.reconstruction_layer = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         # Forward pass through the Encoder
        
#         _, (hidden, _) = self.encoder(x)
        
#         # Concatenate the hidden states of both directions
#         hidden = torch.cat((hidden[0], hidden[1]), dim=1)

#         # Pass the concatenated vector to the reconstruction layer
#         reconstructed_x = self.reconstruction_layer(hidden)

#         return reconstructed_x

# def train_word_encoder(encoder_model, train_loader, criterion, optimizer, num_epochs=10):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     encoder_model.to(device)

#     for epoch in range(num_epochs):
#         encoder_model.train()
        
#         for batch_idx, (data, target) in enumerate(train_loader):
#             # Move the data to the device that is used
#             data = data.to(device)

#             # Forward pass the Encoder
#             output = encoder_model(data)
            
#             # Calculate the batch loss
#             loss = criterion(output, target)
            
#             # Backpropagation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             # Print loss for progress
#             if batch_idx % 100 == 0:
#                 print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))


# # Example usage
# input_size = 300  # Adjust based on the dimension of your word embeddings
# hidden_size = 128
# output_size = 300  # Adjust based on the number of classes for diacritics or the dimension of your embeddings

# word_encoder_model = WordEncoder(input_size, hidden_size, output_size)

# # Define your DataLoader for training, including the input data

# criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
# optimizer = optim.Adam(word_encoder_model.parameters(), lr=0.001)

# #load embeddings bin from fasttext
# import fasttext
# import numpy as np
# import torch
# from torch.utils.data import Dataset, DataLoader

# class WordEmbeddingDataset(Dataset):
#     def __init__(self, embeddings_bin_path):
#         self.embeddings = fasttext.load_model(embeddings_bin_path)

#     def __len__(self):
#         return len(self.embeddings.words)

#     def __getitem__(self, idx):
#         word = self.embeddings.words[idx]
#         embedding = torch.tensor(self.embeddings[word])

#         return embedding, word
    
# train_dataset = WordEmbeddingDataset("word_embed_model.bin")

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# train_word_encoder(word_encoder_model, train_loader, criterion, optimizer, num_epochs=10)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import fasttext
class WordEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(WordEncoder, self).__init__()
        self.hidden_size = hidden_size

        # Define the Encoder layer as bidirectional LSTM
        #TODO : FIX the bidirectional LSTM
        self.encoder = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)

        # Define the output layer for reconstruction
        self.reconstruction_layer = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the Encoder
        _, (hidden, _) = self.encoder(x)

        # # Concatenate the hidden states of both directions
        # hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=0)  # Use dim=0 to concatenate along the batch dimension

        # Pass the concatenated vector to the reconstruction layer
        reconstructed_x = self.reconstruction_layer(hidden)

        return reconstructed_x



class WordEmbeddingDataset(Dataset):
    def __init__(self, embeddings_bin_path):
        self.embeddings = fasttext.load_model(embeddings_bin_path)

    def __len__(self):
        return len(self.embeddings.words)

    def __getitem__(self, idx):
        word = self.embeddings.words[idx]
        embedding = torch.tensor(self.embeddings[word])

        return embedding, embedding  # Using the embedding as target for reconstruction

def train_word_encoder(encoder_model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    encoder_model.to(device)

    for epoch in range(num_epochs):
        encoder_model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            # Move the data to the device that is used
            data, target = data.to(device), target.to(device)

            # Forward pass the Encoder
            output = encoder_model(data)

            # Calculate the batch loss
            loss = criterion(output, target)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print loss for progress
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

# Example usage
input_size = 300  # Adjust based on the dimension of your word embeddings
hidden_size = 128
output_size = 300  # Adjust based on the number of classes for diacritics or the dimension of your embeddings

word_encoder_model = WordEncoder(input_size, hidden_size, output_size)

# Define your DataLoader for training, including the input data
train_dataset = WordEmbeddingDataset("word_embed_model.bin")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
optimizer = optim.Adam(word_encoder_model.parameters(), lr=0.001)

train_word_encoder(word_encoder_model, train_loader, criterion, optimizer, num_epochs=10)
